In [1]:
import json
import os
import warnings
warnings.filterwarnings(action='ignore')

from tqdm import tqdm
from glob import glob

import numpy as np
import pandas as pd

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

In [2]:
import pycocotools
pycocotools.__path__

['/home/kos/anaconda3/envs/PhD/lib/python3.8/site-packages/pycocotools']

## RES vs SW - different ROI placement and filtering methods

In [22]:
# th = 0.2
th = 0.01

subdirs = [
    # 'baseline/DC-val-sw',
    # 'baseline/DC-val-resize',
    # 'move-roi/DC-val-sw',
    # 'move-roi/DC-val-resize',
    # 'move-roi-extend-sw/DC-val-sw',
    # 'move-roi-extend-sw/DC-val-resize',
    # 'move-roi-extend-sw-move-sw/DC-val-sw',
    # 'move-roi-extend-sw-move-sw/DC-val-resize',
    'move-roi-extend-sw-move-sw-fix-filter-ul/DC-val-sw',
    'move-roi-extend-sw-move-sw-fix-filter-ul/DC-val-resize',
]

all_metrics = []
for subdir in subdirs:
    detections = [
        f'ablation/fix-res-sw/{subdir}/results-val.json', 
    ]

    gts = [
        'data/DroneCrowd/val.json',
    ]


    names = ['method', 'dataset', 'AP', 'AP50', 'AP75', 'APu', 'APvt', 'APt', 'APs', 'APm', 'APl', 'AR500', 'ARu', 'ARvt', 'ARt', 'ARs', 'ARm', 'ARl']
    max_dets = [500]
    iou_th = 0.5

    
    for det_path, gt_path in zip(detections, gts):
        if not os.path.isfile(det_path):
            print(det_path)
            continue
            
        
            
        det = json.load(open(det_path))
        print(len(det))
        det = [x for x in det if x['score'] >= th] if th is not None else det
        print(len(det))
        print(max_dets, iou_th)
        
        th_path = det_path.replace('.json', f'-mapped.json')
        with open(th_path, 'w', encoding='utf-8') as f:
            json.dump(det, f, ensure_ascii=False, indent=4)
    
        method, ds = os.path.dirname(det_path).split(os.sep)[-2:]
        anno = COCO(gt_path)

        pred = anno.loadRes(th_path)  # init predictions api
        
        eval = COCOeval(anno, pred, 'bbox', iouThr=iou_th, maxDets=max_dets)

        imgIds = sorted(anno.getImgIds())
        eval.params.imgIds = imgIds

        eval.evaluate()
        eval.accumulate()
        eval.summarize()
        map, map50 = eval.stats[:2]

        stats = {k:v for k,v in zip(names, [method, ds] + [x*100 for x in eval.stats.tolist()])}
        all_metrics.append(stats)

metrics = pd.DataFrame(all_metrics)

row = metrics.iloc[0].values.tolist()
indices = list(filter(lambda x: row[x] == -100, range(len(row))))
columns = [x for i,x in enumerate(metrics.columns.tolist()) if i not in indices]
metrics = metrics[columns]

metrics.to_csv(f'ablation/fix-res-sw/move-roi-extend-sw-fix-filter-ul.csv', index=False)

143197
143197
[500] 0.5
loading annotations into memory...
Done (t=0.41s)
creating index...
index created!
Loading and preparing results...
DONE (t=2.86s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=385.67s).
Accumulating evaluation results...
DONE (t=0.51s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.224
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.544
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.136
 Average Precision  (AP) @[ IoU=0.50      | area= micro | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=v-tiny | maxDets=500 ] = 0.547
 Average Precision  (AP) @[ IoU=0.50      | area=  tiny | maxDets=500 ] = 0.892
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50

In [23]:
metrics

,method,dataset,AP,AP50,AP75,APvt,APt,AR500,ARvt,ARt
0,move-roi-extend-sw-move-sw-fix-filter-ul,DC-val-sw,22.394476,54.377641,13.551921,54.659582,89.19568,70.629128,70.63947,100.0


In [21]:
metrics

,method,dataset,AP,AP50,AP75,APvt,APt,AR500,ARvt,ARt
0,move-roi-extend-sw-move-sw-fix-filter-full,DC-val-sw,23.287769,55.821246,13.929133,56.289508,76.851708,72.894072,72.905660,100.000000
1,move-roi-extend-sw-move-sw-fix-filter-full,DC-val-resize,17.942007,49.213683,7.963895,53.944512,5.263622,66.612982,66.617032,92.592593


In [ ]:
metrics

,method,dataset,AP,AP50,AP75,APvt,APt,AR500,ARvt,ARt
0,baseline,DC-val-sw,23.380167,56.949112,13.907655,57.394061,80.060245,73.491397,73.503315,100.000000
1,baseline,DC-val-resize,18.227028,50.194847,8.062642,55.116506,5.566199,67.041099,67.053544,92.592593


# DC val - different models

In [26]:
# th = 0.2
th = 0.01

subdirs = [
    'models/001-320x512',
    'models/001-512x512',
    'models/003-512x512',
    'models/004-512x512',
    'models/005-512x512',
    'models/006-512x512',
]

all_metrics = []
for subdir in subdirs:
    detections = [
        f'ablation/soa-dc/{subdir}/results-val.json', 
    ]

    gts = [
        'data/DroneCrowd/val.json',
    ]


    names = ['method', 'dataset', 'AP', 'AP50', 'AP75', 'APu', 'APvt', 'APt', 'APs', 'APm', 'APl', 'AR500', 'ARu', 'ARvt', 'ARt', 'ARs', 'ARm', 'ARl']
    max_dets = [500]
    iou_th = 0.5

    
    for det_path, gt_path in zip(detections, gts):
        if not os.path.isfile(det_path):
            print(det_path)
            continue
            
        
            
        det = json.load(open(det_path))
        print(len(det))
        det = [x for x in det if x['score'] >= th] if th is not None else det
        print(len(det))
        print(max_dets, iou_th)
        
        th_path = det_path.replace('.json', f'-mapped.json')
        with open(th_path, 'w', encoding='utf-8') as f:
            json.dump(det, f, ensure_ascii=False, indent=4)
    
        method, ds = os.path.dirname(det_path).split(os.sep)[-2:]
        anno = COCO(gt_path)

        pred = anno.loadRes(th_path)  # init predictions api
        
        eval = COCOeval(anno, pred, 'bbox', iouThr=iou_th, maxDets=max_dets)

        imgIds = sorted(anno.getImgIds())
        eval.params.imgIds = imgIds

        eval.evaluate()
        eval.accumulate()
        eval.summarize()
        map, map50 = eval.stats[:2]

        stats = {k:v for k,v in zip(names, [method, ds] + [x*100 for x in eval.stats.tolist()])}
        all_metrics.append(stats)

metrics = pd.DataFrame(all_metrics)

row = metrics.iloc[0].values.tolist()
indices = list(filter(lambda x: row[x] == -100, range(len(row))))
columns = [x for i,x in enumerate(metrics.columns.tolist()) if i not in indices]
metrics = metrics[columns]

metrics.to_csv(f'ablation/soa-dc/models-val.csv', index=False)

91287
91287
[500] 0.5
loading annotations into memory...
Done (t=0.36s)
creating index...
index created!
Loading and preparing results...
DONE (t=2.16s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=331.48s).
Accumulating evaluation results...
DONE (t=0.47s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.213
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.550
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.106
 Average Precision  (AP) @[ IoU=0.50      | area= micro | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=v-tiny | maxDets=500 ] = 0.553
 Average Precision  (AP) @[ IoU=0.50      | area=  tiny | maxDets=500 ] = 0.885
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50  

In [27]:
metrics

,method,dataset,AP,AP50,AP75,APvt,APt,AR500,ARvt,ARt
0,models,001-320x512,21.256069,55.037462,10.626587,55.262883,88.499453,67.346897,67.355431,100.000000
1,models,001-512x512,21.454766,54.830622,11.387816,55.075753,80.219139,66.857620,66.867925,92.592593
2,models,003-512x512,20.803461,53.489583,10.217407,55.418835,53.395735,66.615021,66.631311,100.000000
3,models,004-512x512,21.697709,55.353525,11.617576,55.474296,92.421752,69.901329,69.913310,100.000000
4,models,005-512x512,20.529765,54.622686,9.445080,58.637012,28.009017,68.382533,68.401836,96.296296
5,models,006-512x512,21.384417,55.753071,10.562697,55.942427,85.969326,70.663785,70.680265,92.592593


# 004 model + SEG vs SEG_TRACK vs different aggregation methods

In [42]:
# th = 0.2
th = 0.01

subdirs = [
    # 'det-windows/before_resize/seg',
    # 'det-windows/before_resize/seg_track',
    # 'det-windows/fixes_resize/seg',
    # 'det-windows/fixes_resize/seg_track',
    # 'det-windows/fixes_resize_noext/seg',
    # 'det-windows/fixes_resize_noext/seg_track',
    # 'det-windows/fixes_resize_noextx2/seg',
    # 'det-windows/fixes_resize_noextx2/seg_track',
    # 'det-windows/fixes_resize_ul/seg',
    # 'det-windows/fixes_resize_ul/seg_track',
    'det-windows/fixes_resize_noextx2_noNMS/seg',
    'det-windows/fixes_resize_noextx2_noNMS/seg_track',
]

all_metrics = []
for subdir in subdirs:
    detections = [
        f'ablation/soa-dc/{subdir}/results-val.json', 
    ]

    gts = [
        'data/DroneCrowd/val.json',
    ]


    names = ['method', 'dataset', 'AP', 'AP50', 'AP75', 'APu', 'APvt', 'APt', 'APs', 'APm', 'APl', 'AR500', 'ARu', 'ARvt', 'ARt', 'ARs', 'ARm', 'ARl']
    max_dets = [500]
    iou_th = 0.5

    
    for det_path, gt_path in zip(detections, gts):
        if not os.path.isfile(det_path):
            print(det_path)
            continue
            
        
            
        det = json.load(open(det_path))
        print(len(det))
        det = [x for x in det if x['score'] >= th] if th is not None else det
        print(len(det))
        print(max_dets, iou_th)
        
        th_path = det_path.replace('.json', f'-mapped.json')
        with open(th_path, 'w', encoding='utf-8') as f:
            json.dump(det, f, ensure_ascii=False, indent=4)
    
        method, ds = os.path.dirname(det_path).split(os.sep)[-2:]
        anno = COCO(gt_path)

        pred = anno.loadRes(th_path)  # init predictions api
        
        eval = COCOeval(anno, pred, 'bbox', iouThr=iou_th, maxDets=max_dets)

        imgIds = sorted(anno.getImgIds())
        eval.params.imgIds = imgIds

        eval.evaluate()
        eval.accumulate()
        eval.summarize()
        map, map50 = eval.stats[:2]

        stats = {k:v for k,v in zip(names, [method, ds] + [x*100 for x in eval.stats.tolist()])}
        all_metrics.append(stats)

metrics = pd.DataFrame(all_metrics)

row = metrics.iloc[0].values.tolist()
indices = list(filter(lambda x: row[x] == -100, range(len(row))))
columns = [x for i,x in enumerate(metrics.columns.tolist()) if i not in indices]
metrics = metrics[columns]

metrics = pd.concat([metrics, pd.read_csv(f'ablation/soa-dc/det-windows-seg-track-val.csv')], ignore_index=True)
metrics.to_csv(f'ablation/soa-dc/det-windows-seg-track-val.csv', index=False)

147084
147084
[500] 0.5
loading annotations into memory...
Done (t=0.58s)
creating index...
index created!
Loading and preparing results...
DONE (t=4.36s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=334.62s).
Accumulating evaluation results...
DONE (t=0.73s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.227
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.563
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.127
 Average Precision  (AP) @[ IoU=0.50      | area= micro | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=v-tiny | maxDets=500 ] = 0.565
 Average Precision  (AP) @[ IoU=0.50      | area=  tiny | maxDets=500 ] = 0.878
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50

In [43]:
metrics

,method,dataset,AP,AP50,AP75,APvt,APt,AR500,ARvt,ARt
0,fixes_resize_noextx2_noNMS,seg,22.717571,56.274831,12.720047,56.493684,87.828566,73.617793,73.629781,100.000000
1,fixes_resize_noextx2_noNMS,seg_track,22.567913,56.119821,12.597112,56.355431,90.189557,73.774770,73.786843,100.000000
2,before_resize,seg,21.697709,55.353525,11.617576,55.474296,92.421752,69.901329,69.913310,100.000000
3,before_resize,seg_track,21.775018,55.429349,11.729904,55.551047,88.736508,69.880943,69.894952,96.296296
4,fixes_resize,seg,21.373566,54.956972,11.095572,55.127282,92.175884,68.091005,68.101989,96.296296
5,fixes_resize,seg_track,21.414321,54.989998,11.029236,55.163295,92.047564,68.158281,68.167262,100.000000
6,fixes_resize_noext,seg,21.857868,55.813178,11.552035,55.966273,92.188073,69.601647,69.615502,96.296296
7,fixes_resize_noext,seg_track,21.828968,55.781925,11.423108,55.931062,92.237781,69.624072,69.633860,100.000000
8,fixes_resize_noextx2,seg,21.644836,55.265616,11.390990,55.400328,91.997818,69.383511,69.397246,96.296296
9,fixes_resize_noextx2,seg_track,21.640181,55.299533,11.403267,55.436019,92.590640,69.526217,69.537991,100.000000


In [46]:
metrics.sort_values(by='AP50', ascending=False)

,method,dataset,AP,AP50,AP75,APvt,APt,AR500,ARvt,ARt
0,fixes_resize_noextx2_noNMS,seg,22.717571,56.274831,12.720047,56.493684,87.828566,73.617793,73.629781,100.000000
1,fixes_resize_noextx2_noNMS,seg_track,22.567913,56.119821,12.597112,56.355431,90.189557,73.774770,73.786843,100.000000
6,fixes_resize_noext,seg,21.857868,55.813178,11.552035,55.966273,92.188073,69.601647,69.615502,96.296296
7,fixes_resize_noext,seg_track,21.828968,55.781925,11.423108,55.931062,92.237781,69.624072,69.633860,100.000000
3,before_resize,seg_track,21.775018,55.429349,11.729904,55.551047,88.736508,69.880943,69.894952,96.296296
2,before_resize,seg,21.697709,55.353525,11.617576,55.474296,92.421752,69.901329,69.913310,100.000000
9,fixes_resize_noextx2,seg_track,21.640181,55.299533,11.403267,55.436019,92.590640,69.526217,69.537991,100.000000
8,fixes_resize_noextx2,seg,21.644836,55.265616,11.390990,55.400328,91.997818,69.383511,69.397246,96.296296
5,fixes_resize,seg_track,21.414321,54.989998,11.029236,55.163295,92.047564,68.158281,68.167262,100.000000
4,fixes_resize,seg,21.373566,54.956972,11.095572,55.127282,92.175884,68.091005,68.101989,96.296296


In [ ]:
noext, before, noextx2 - x3 (AP, AP50, AP75) - nowxt x2 ma najwięcej sensu
najwyżej zmiana na before resize, gdy będzie brakowało na teście

# DC - 004 + different val res + OBS 0.7 (noextx2)

In [48]:
# th = 0.2
th = 0.01

subdirs = [
    # 'seg-res/noext-roi-tiny',
    # 'seg-res/noext-roi-small',
    # 'seg-res/noext-roi-medium',
    # 'seg-res/noext-roi_track-tiny',
    # 'seg-res/noext-roi_track-small',
    # 'seg-res/noext-roi_track-medium',
    'seg-res/noext-roi-medium-th05',
    'seg-res/noext-roi_track-medium-th05'
]

all_metrics = []
for subdir in subdirs:
    detections = [
        f'ablation/soa-dc/{subdir}/results-val.json', 
    ]

    gts = [
        'data/DroneCrowd/val.json',
    ]


    names = ['method', 'dataset', 'AP', 'AP50', 'AP75', 'APu', 'APvt', 'APt', 'APs', 'APm', 'APl', 'AR500', 'ARu', 'ARvt', 'ARt', 'ARs', 'ARm', 'ARl']
    max_dets = [500]
    iou_th = 0.5

    
    for det_path, gt_path in zip(detections, gts):
        if not os.path.isfile(det_path):
            print(det_path)
            continue
            
        
            
        det = json.load(open(det_path))
        print(len(det))
        det = [x for x in det if x['score'] >= th] if th is not None else det
        print(len(det))
        print(max_dets, iou_th)
        
        th_path = det_path.replace('.json', f'-mapped.json')
        with open(th_path, 'w', encoding='utf-8') as f:
            json.dump(det, f, ensure_ascii=False, indent=4)
    
        method, ds = os.path.dirname(det_path).split(os.sep)[-2:]
        anno = COCO(gt_path)

        pred = anno.loadRes(th_path)  # init predictions api
        
        eval = COCOeval(anno, pred, 'bbox', iouThr=iou_th, maxDets=max_dets)

        imgIds = sorted(anno.getImgIds())
        eval.params.imgIds = imgIds

        eval.evaluate()
        eval.accumulate()
        eval.summarize()
        map, map50 = eval.stats[:2]

        stats = {k:v for k,v in zip(names, [method, ds] + [x*100 for x in eval.stats.tolist()])}
        all_metrics.append(stats)

metrics = pd.DataFrame(all_metrics)

row = metrics.iloc[0].values.tolist()
indices = list(filter(lambda x: row[x] == -100, range(len(row))))
columns = [x for i,x in enumerate(metrics.columns.tolist()) if i not in indices]
metrics = metrics[columns]

metrics.to_csv(f'ablation/soa-dc/res-res-val.csv', index=False)ignore_index=

42705
42705
[500] 0.5
loading annotations into memory...
Done (t=0.84s)
creating index...
index created!
Loading and preparing results...
DONE (t=1.31s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=143.16s).
Accumulating evaluation results...
DONE (t=0.33s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.056
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.147
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.028
 Average Precision  (AP) @[ IoU=0.50      | area= micro | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=v-tiny | maxDets=500 ] = 0.148
 Average Precision  (AP) @[ IoU=0.50      | area=  tiny | maxDets=500 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50  

In [49]:
metrics

,method,dataset,AP,AP50,AP75,APvt,APt,AR500,ARvt,ARt
0,seg-res,noext-roi-tiny,5.594062,14.747849,2.789283,14.801231,0.000000,20.270325,20.281489,0.0
1,seg-res,noext-roi-small,22.469054,55.404850,13.170385,55.547217,93.070006,72.045992,72.057114,100.0
2,seg-res,noext-roi-medium,23.214657,57.221341,13.344525,57.487835,72.991003,75.921471,75.934727,100.0
3,seg-res,noext-roi_track-tiny,7.700095,20.338218,4.177824,20.403205,0.000000,28.512599,28.528302,0.0
4,seg-res,noext-roi_track-small,23.273417,56.666858,13.539065,56.819421,88.703611,74.276278,74.288628,100.0
5,seg-res,noext-roi_track-medium,23.250983,56.965933,13.284217,57.222520,79.320104,75.846041,75.857216,100.0


In [51]:
# th = 0.2
th = 0.01

subdirs = [
    # 'seg-res/noext-roi-tiny',
    # 'seg-res/noext-roi-small',
    # 'seg-res/noext-roi-medium',
    # 'seg-res/noext-roi_track-tiny',
    # 'seg-res/noext-roi_track-small',
    # 'seg-res/noext-roi_track-medium',
    'seg-res/noext-roi-medium-th05',  # no dilate th 0.5
    'seg-res/noext-roi_track-medium-th05', # no dilate th 0.5
    'det-windows/fixes_resize_noextx2_noNMS/seg', # dilate th 0.5
    'det-windows/fixes_resize_noextx2_noNMS/seg_track', # dilate th 0.5
]

all_metrics = []
for subdir in subdirs:
    detections = [
        f'ablation/soa-dc/{subdir}/results-val.json', 
    ]

    gts = [
        'data/DroneCrowd/val.json',
    ]


    names = ['method', 'dataset', 'AP', 'AP50', 'AP75', 'APu', 'APvt', 'APt', 'APs', 'APm', 'APl', 'AR500', 'ARu', 'ARvt', 'ARt', 'ARs', 'ARm', 'ARl']
    max_dets = [500]
    iou_th = 0.5

    
    for det_path, gt_path in zip(detections, gts):
        if not os.path.isfile(det_path):
            print(det_path)
            continue
            
        
            
        det = json.load(open(det_path))
        print(len(det))
        det = [x for x in det if x['score'] >= th] if th is not None else det
        print(len(det))
        print(max_dets, iou_th)
        
        th_path = det_path.replace('.json', f'-mapped.json')
        with open(th_path, 'w', encoding='utf-8') as f:
            json.dump(det, f, ensure_ascii=False, indent=4)
    
        method, ds = os.path.dirname(det_path).split(os.sep)[-2:]
        anno = COCO(gt_path)

        pred = anno.loadRes(th_path)  # init predictions api
        
        eval = COCOeval(anno, pred, 'bbox', iouThr=iou_th, maxDets=max_dets)

        imgIds = sorted(anno.getImgIds())
        eval.params.imgIds = imgIds

        eval.evaluate()
        eval.accumulate()
        eval.summarize()
        map, map50 = eval.stats[:2]

        stats = {k:v for k,v in zip(names, [method, ds] + [x*100 for x in eval.stats.tolist()])}
        all_metrics.append(stats)

metrics2 = pd.DataFrame(all_metrics)

row = metrics2.iloc[0].values.tolist()
indices = list(filter(lambda x: row[x] == -100, range(len(row))))
columns = [x for i,x in enumerate(metrics2.columns.tolist()) if i not in indices]
metrics2 = metrics2[columns]

metrics = pd.concat([pd.read_csv(f'ablation/soa-dc/res-res-val.csv'), metrics2], ignore_index=True)
metrics

144297
144297
[500] 0.5
loading annotations into memory...
Done (t=0.12s)
creating index...
index created!
Loading and preparing results...
DONE (t=4.94s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=358.75s).
Accumulating evaluation results...
DONE (t=0.84s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.228
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.568
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.128
 Average Precision  (AP) @[ IoU=0.50      | area= micro | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=v-tiny | maxDets=500 ] = 0.570
 Average Precision  (AP) @[ IoU=0.50      | area=  tiny | maxDets=500 ] = 0.730
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50

,method,dataset,AP,AP50,AP75,APvt,APt,AR500,ARvt,ARt
0,seg-res,noext-roi-tiny,5.594062,14.747849,2.789283,14.801231,0.000000,20.270325,20.281489,0.0
1,seg-res,noext-roi-small,22.469054,55.404850,13.170385,55.547217,93.070006,72.045992,72.057114,100.0
2,seg-res,noext-roi-medium,23.214657,57.221341,13.344525,57.487835,72.991003,75.921471,75.934727,100.0
3,seg-res,noext-roi_track-tiny,7.700095,20.338218,4.177824,20.403205,0.000000,28.512599,28.528302,0.0
4,seg-res,noext-roi_track-small,23.273417,56.666858,13.539065,56.819421,88.703611,74.276278,74.288628,100.0
5,seg-res,noext-roi_track-medium,23.250983,56.965933,13.284217,57.222520,79.320104,75.846041,75.857216,100.0
6,seg-res,noext-roi-medium-th05,22.756061,56.768019,12.810677,57.046506,72.996933,74.023485,74.035696,100.0
7,seg-res,noext-roi_track-medium-th05,22.668334,56.105655,12.886808,56.368986,75.980985,73.935823,73.945946,100.0
8,fixes_resize_noextx2_noNMS,seg,22.717571,56.274831,12.720047,56.493684,87.828566,73.617793,73.629781,100.0
9,fixes_resize_noextx2_noNMS,seg_track,22.567913,56.119821,12.597112,56.355431,90.189557,73.774770,73.786843,100.0


In [52]:
metrics.to_csv(f'ablation/soa-dc/res-res-val.csv', index=False)

In [53]:
# th = 0.2
th = 0.01

subdirs = [
    # 'seg-res/noext-roi-tiny',
    # 'seg-res/noext-roi-small',
    # 'seg-res/noext-roi-medium',
    # 'seg-res/noext-roi_track-tiny',
    # 'seg-res/noext-roi_track-small',
    # 'seg-res/noext-roi_track-medium',
    'seg-res/noext-roi-small_dil',  
    'seg-res/noext-roi_track-small_dil', 
    'seg-res/beforefixres-roi-small_dil',
    'seg-res/beforefixres-roi_track-small_dil', 
]

all_metrics = []
for subdir in subdirs:
    detections = [
        f'ablation/soa-dc/{subdir}/results-val.json', 
    ]

    gts = [
        'data/DroneCrowd/val.json',
    ]


    names = ['method', 'dataset', 'AP', 'AP50', 'AP75', 'APu', 'APvt', 'APt', 'APs', 'APm', 'APl', 'AR500', 'ARu', 'ARvt', 'ARt', 'ARs', 'ARm', 'ARl']
    max_dets = [500]
    iou_th = 0.5

    
    for det_path, gt_path in zip(detections, gts):
        if not os.path.isfile(det_path):
            print(det_path)
            continue
            
        
            
        det = json.load(open(det_path))
        print(len(det))
        det = [x for x in det if x['score'] >= th] if th is not None else det
        print(len(det))
        print(max_dets, iou_th)
        
        th_path = det_path.replace('.json', f'-mapped.json')
        with open(th_path, 'w', encoding='utf-8') as f:
            json.dump(det, f, ensure_ascii=False, indent=4)
    
        method, ds = os.path.dirname(det_path).split(os.sep)[-2:]
        anno = COCO(gt_path)

        pred = anno.loadRes(th_path)  # init predictions api
        
        eval = COCOeval(anno, pred, 'bbox', iouThr=iou_th, maxDets=max_dets)

        imgIds = sorted(anno.getImgIds())
        eval.params.imgIds = imgIds

        eval.evaluate()
        eval.accumulate()
        eval.summarize()
        map, map50 = eval.stats[:2]

        stats = {k:v for k,v in zip(names, [method, ds] + [x*100 for x in eval.stats.tolist()])}
        all_metrics.append(stats)

metrics2 = pd.DataFrame(all_metrics)

row = metrics2.iloc[0].values.tolist()
indices = list(filter(lambda x: row[x] == -100, range(len(row))))
columns = [x for i,x in enumerate(metrics2.columns.tolist()) if i not in indices]
metrics2 = metrics2[columns]

metrics = pd.concat([pd.read_csv(f'ablation/soa-dc/res-res-val.csv'), metrics2], ignore_index=True)
metrics

151409
151409
[500] 0.5
loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
Loading and preparing results...
DONE (t=5.51s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=431.94s).
Accumulating evaluation results...
DONE (t=0.75s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.220
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.539
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.130
 Average Precision  (AP) @[ IoU=0.50      | area= micro | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=v-tiny | maxDets=500 ] = 0.541
 Average Precision  (AP) @[ IoU=0.50      | area=  tiny | maxDets=500 ] = 0.911
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50

KeyboardInterrupt: 

In [54]:
metrics2 = pd.DataFrame(all_metrics)

row = metrics2.iloc[0].values.tolist()
indices = list(filter(lambda x: row[x] == -100, range(len(row))))
columns = [x for i,x in enumerate(metrics2.columns.tolist()) if i not in indices]
metrics2 = metrics2[columns]

metrics = pd.concat([pd.read_csv(f'ablation/soa-dc/res-res-val.csv'), metrics2], ignore_index=True)
metrics

,method,dataset,AP,AP50,AP75,APvt,APt,AR500,ARvt,ARt
0,seg-res,noext-roi-tiny,5.594062,14.747849,2.789283,14.801231,0.000000,20.270325,20.281489,0.0
1,seg-res,noext-roi-small,22.469054,55.404850,13.170385,55.547217,93.070006,72.045992,72.057114,100.0
2,seg-res,noext-roi-medium,23.214657,57.221341,13.344525,57.487835,72.991003,75.921471,75.934727,100.0
3,seg-res,noext-roi_track-tiny,7.700095,20.338218,4.177824,20.403205,0.000000,28.512599,28.528302,0.0
4,seg-res,noext-roi_track-small,23.273417,56.666858,13.539065,56.819421,88.703611,74.276278,74.288628,100.0
5,seg-res,noext-roi_track-medium,23.250983,56.965933,13.284217,57.222520,79.320104,75.846041,75.857216,100.0
6,seg-res,noext-roi-medium-th05,22.756061,56.768019,12.810677,57.046506,72.996933,74.023485,74.035696,100.0
7,seg-res,noext-roi_track-medium-th05,22.668334,56.105655,12.886808,56.368986,75.980985,73.935823,73.945946,100.0
8,fixes_resize_noextx2_noNMS,seg,22.717571,56.274831,12.720047,56.493684,87.828566,73.617793,73.629781,100.0
9,fixes_resize_noextx2_noNMS,seg_track,22.567913,56.119821,12.597112,56.355431,90.189557,73.774770,73.786843,100.0


In [55]:
metrics.to_csv(f'ablation/soa-dc/res-res-val.csv', index=False)

In [56]:
# th = 0.2
th = 0.01

subdirs = [
    'seg-res/beforefixres-roi-small_nodil',  
    'seg-res/beforefixres-roi_track-small_nodil', 
]

all_metrics = []
for subdir in subdirs:
    detections = [
        f'ablation/soa-dc/{subdir}/results-val.json', 
    ]

    gts = [
        'data/DroneCrowd/val.json',
    ]


    names = ['method', 'dataset', 'AP', 'AP50', 'AP75', 'APu', 'APvt', 'APt', 'APs', 'APm', 'APl', 'AR500', 'ARu', 'ARvt', 'ARt', 'ARs', 'ARm', 'ARl']
    max_dets = [500]
    iou_th = 0.5

    
    for det_path, gt_path in zip(detections, gts):
        if not os.path.isfile(det_path):
            print(det_path)
            continue
            
        
            
        det = json.load(open(det_path))
        print(len(det))
        det = [x for x in det if x['score'] >= th] if th is not None else det
        print(len(det))
        print(max_dets, iou_th)
        
        th_path = det_path.replace('.json', f'-mapped.json')
        with open(th_path, 'w', encoding='utf-8') as f:
            json.dump(det, f, ensure_ascii=False, indent=4)
    
        method, ds = os.path.dirname(det_path).split(os.sep)[-2:]
        anno = COCO(gt_path)

        pred = anno.loadRes(th_path)  # init predictions api
        
        eval = COCOeval(anno, pred, 'bbox', iouThr=iou_th, maxDets=max_dets)

        imgIds = sorted(anno.getImgIds())
        eval.params.imgIds = imgIds

        eval.evaluate()
        eval.accumulate()
        eval.summarize()
        map, map50 = eval.stats[:2]

        stats = {k:v for k,v in zip(names, [method, ds] + [x*100 for x in eval.stats.tolist()])}
        all_metrics.append(stats)

metrics2 = pd.DataFrame(all_metrics)

row = metrics2.iloc[0].values.tolist()
indices = list(filter(lambda x: row[x] == -100, range(len(row))))
columns = [x for i,x in enumerate(metrics2.columns.tolist()) if i not in indices]
metrics2 = metrics2[columns]

metrics = pd.concat([pd.read_csv(f'ablation/soa-dc/res-res-val.csv'), metrics2], ignore_index=True)
metrics

162921
162921
[500] 0.5
loading annotations into memory...
Done (t=0.14s)
creating index...
index created!
Loading and preparing results...
DONE (t=10.22s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=456.69s).
Accumulating evaluation results...
DONE (t=0.76s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.225
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.553
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.130
 Average Precision  (AP) @[ IoU=0.50      | area= micro | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=v-tiny | maxDets=500 ] = 0.554
 Average Precision  (AP) @[ IoU=0.50      | area=  tiny | maxDets=500 ] = 0.923
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.5

,method,dataset,AP,AP50,AP75,APvt,APt,AR500,ARvt,ARt
0,seg-res,noext-roi-tiny,5.594062,14.747849,2.789283,14.801231,0.000000,20.270325,20.281489,0.0
1,seg-res,noext-roi-small,22.469054,55.404850,13.170385,55.547217,93.070006,72.045992,72.057114,100.0
2,seg-res,noext-roi-medium,23.214657,57.221341,13.344525,57.487835,72.991003,75.921471,75.934727,100.0
3,seg-res,noext-roi_track-tiny,7.700095,20.338218,4.177824,20.403205,0.000000,28.512599,28.528302,0.0
4,seg-res,noext-roi_track-small,23.273417,56.666858,13.539065,56.819421,88.703611,74.276278,74.288628,100.0
5,seg-res,noext-roi_track-medium,23.250983,56.965933,13.284217,57.222520,79.320104,75.846041,75.857216,100.0
6,seg-res,noext-roi-medium-th05,22.756061,56.768019,12.810677,57.046506,72.996933,74.023485,74.035696,100.0
7,seg-res,noext-roi_track-medium-th05,22.668334,56.105655,12.886808,56.368986,75.980985,73.935823,73.945946,100.0
8,fixes_resize_noextx2_noNMS,seg,22.717571,56.274831,12.720047,56.493684,87.828566,73.617793,73.629781,100.0
9,fixes_resize_noextx2_noNMS,seg_track,22.567913,56.119821,12.597112,56.355431,90.189557,73.774770,73.786843,100.0


# test - DC

In [58]:
sorted(glob(f'ablation/soa-dc/test/**/results-test_minus_val.json'))

['ablation/soa-dc/test/seg-noNMS-obs05-beforeres/results-test_minus_val.json',
 'ablation/soa-dc/test/seg-noNMS-obs05-noext/results-test_minus_val.json',
 'ablation/soa-dc/test/seg_track-noNMS-obs05-beforeres/results-test_minus_val.json',
 'ablation/soa-dc/test/seg_track-noNMS-obs05-noext/results-test_minus_val.json']

In [47]:
# th = 0.2
th = 0.01

subdirs = [
    'seg-noNMS-obs05-noext',
    'seg_track-noNMS-obs05-noext',
]

all_metrics = []
for subdir in subdirs:
    detections = [
        f'ablation/soa-dc/{subdir}/results-test_minus_val.json', 
    ]

    gts = [
        'data/DroneCrowd/test_minus_val.json',
    ]


    names = ['method', 'dataset', 'AP', 'AP50', 'AP75', 'APu', 'APvt', 'APt', 'APs', 'APm', 'APl', 'AR500', 'ARu', 'ARvt', 'ARt', 'ARs', 'ARm', 'ARl']
    max_dets = [500]
    iou_th = 0.5

    
    for det_path, gt_path in zip(detections, gts):
        if not os.path.isfile(det_path):
            print(det_path)
            continue
            
        
            
        det = json.load(open(det_path))
        print(len(det))
        det = [x for x in det if x['score'] >= th] if th is not None else det
        print(len(det))
        print(max_dets, iou_th)
        
        th_path = det_path.replace('.json', f'-mapped.json')
        with open(th_path, 'w', encoding='utf-8') as f:
            json.dump(det, f, ensure_ascii=False, indent=4)
    
        method, ds = os.path.dirname(det_path).split(os.sep)[-2:]
        anno = COCO(gt_path)

        pred = anno.loadRes(th_path)  # init predictions api
        
        eval = COCOeval(anno, pred, 'bbox', iouThr=iou_th, maxDets=max_dets)

        imgIds = sorted(anno.getImgIds())
        eval.params.imgIds = imgIds

        eval.evaluate()
        eval.accumulate()
        eval.summarize()
        map, map50 = eval.stats[:2]

        stats = {k:v for k,v in zip(names, [method, ds] + [x*100 for x in eval.stats.tolist()])}
        all_metrics.append(stats)

metrics = pd.DataFrame(all_metrics)

row = metrics.iloc[0].values.tolist()
indices = list(filter(lambda x: row[x] == -100, range(len(row))))
columns = [x for i,x in enumerate(metrics.columns.tolist()) if i not in indices]
metrics = metrics[columns]

metrics.to_csv(f'ablation/soa-dc/dc-test.csv', index=False)
metrics

3561596
3561596
[500] 0.5
loading annotations into memory...
Done (t=4.98s)
creating index...
index created!
Loading and preparing results...
DONE (t=2842.51s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=7475.15s).
Accumulating evaluation results...
DONE (t=21.37s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.202
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.494
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.124
 Average Precision  (AP) @[ IoU=0.50      | area= micro | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=v-tiny | maxDets=500 ] = 0.498
 Average Precision  (AP) @[ IoU=0.50      | area=  tiny | maxDets=500 ] = 0.532
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ I

,method,dataset,AP,AP50,AP75,APvt,APt,AR500,ARvt,ARt
0,soa-dc,seg-noNMS-obs05-noext,20.204095,49.410954,12.400506,49.782711,53.236057,68.904122,69.063567,70.558532
1,soa-dc,seg_track-noNMS-obs05-noext,20.170570,49.565776,12.373422,49.698933,52.046467,69.121054,69.285796,70.821990


In [ ]:
# th = 0.2
th = 0.01

subdirs = [
    # 'seg-noNMS-obs05-noext',
    # 'seg_track-noNMS-obs05-noext',
    'seg-noNMS-obs05-beforeres',
    'seg_track-noNMS-obs05-beforeres',
    'seg-noNMS-obs07-beforeres',
    'seg_track-noNMS-obs07-beforeres',
    'seg-noNMS-obs07-noext',
    'seg_track-noNMS-obs07-noext',
]

all_metrics = []
for subdir in subdirs:
    detections = [
        f'ablation/soa-dc/{subdir}/results-test_minus_val.json', 
    ]

    gts = [
        'data/DroneCrowd/test_minus_val.json',
    ]


    names = ['method', 'dataset', 'AP', 'AP50', 'AP75', 'APu', 'APvt', 'APt', 'APs', 'APm', 'APl', 'AR500', 'ARu', 'ARvt', 'ARt', 'ARs', 'ARm', 'ARl']
    max_dets = [500]
    iou_th = 0.5

    
    for det_path, gt_path in zip(detections, gts):
        if not os.path.isfile(det_path):
            print(det_path)
            continue
            
        
            
        det = json.load(open(det_path))
        print(len(det))
        det = [x for x in det if x['score'] >= th] if th is not None else det
        print(len(det))
        print(max_dets, iou_th)
        
        th_path = det_path.replace('.json', f'-mapped.json')
        with open(th_path, 'w', encoding='utf-8') as f:
            json.dump(det, f, ensure_ascii=False, indent=4)
    
        method, ds = os.path.dirname(det_path).split(os.sep)[-2:]
        anno = COCO(gt_path)

        pred = anno.loadRes(th_path)  # init predictions api
        
        eval = COCOeval(anno, pred, 'bbox', iouThr=iou_th, maxDets=max_dets)

        imgIds = sorted(anno.getImgIds())
        eval.params.imgIds = imgIds

        eval.evaluate()
        eval.accumulate()
        eval.summarize()
        map, map50 = eval.stats[:2]

        stats = {k:v for k,v in zip(names, [method, ds] + [x*100 for x in eval.stats.tolist()])}
        all_metrics.append(stats)

metrics = pd.DataFrame(all_metrics)

row = metrics.iloc[0].values.tolist()
indices = list(filter(lambda x: row[x] == -100, range(len(row))))
columns = [x for i,x in enumerate(metrics.columns.tolist()) if i not in indices]
metrics = metrics[columns]

metrics.to_csv(f'ablation/soa-dc/dc-test-v2.csv', index=False) # TODO merge with dc-test.csv
metrics